# Hackathon Stat

This project illustrates the course LEPL1109 with an industrial application of statistics. You will analyse the capacity of solar production of electricity located in the French cities of Caen and Tours.
The file 'radiation.csv' contains 3 columns 
DATE           : 2023-10-09,
Caen and Tours : the daily solar radiation in W/m2 measured in the 2 cities. 
Notice that data for some days are not reported due to failure of measurement system.

## Report content

•	You have to fill in this  jupyter notebook downloadable on the moodle website of the course

•	Grades are granted to the members whose names are in the Jupyter notebook. If your name doesn’t appear on the top of the notebook, you’ll get a 0, even though you are in a group on Moodle.

•	The jupyter notebook must be compiled with printed results and next submitted via moodle. The absence of compiled results (or non-printed values) leads to a lower grade.

## Report submission

•	The deadline for submission is reported on the moodle website. Submission after the deadline will not be accepted.

•	To submit your report, go to the section “APP” on Moodle and the subsection “Soumission du rapport”. You can upload your work there. Once you are sure that it is your final version, click the button “Envoyer le devoir”. It is important that you don’t forget to click on this button ! 

•	Reports that have not been uploaded through Moodle will not be corrected.

## Names and Noma of participants:

Part. 1: Aymeric Wibo 7482-21-00

Part. 2: Emile Van Gysel 1747-21-00

Part. 3: Wuillaume Vincent 9095-21-00

Part. 4: Raphaël Cavenaile 4022-21-00

Part. 5: Englebert Alexis 3786-21-00

Part. 6: Alessandro Giansante 6385-21-00

---
## 1. Energy calculation and basic statistics

Compute the daily energy in Wh per square meter of solar panel. For this purpose you use the datasets reporting the solar irradation measure in Caen and Tours (source https://www.ecad.eu/). The irradiation is measured in W/m2 per day. You will use the formula:

$$C = E_{sol} * 24 * P_{cr} * f_{perf}$$

where  

$C$ is the electricity produced in Wh/m^2 for a day

$E_{sol}$ is the daily solar radiation in W/m^2

$P_{cr}$ is the peak power coefficient, set here to 0.18 (monocristal silicium)

$f_{perf}$ depends upon the system, set here to 0.75.

Remark:

1 W = 1 J/sec

1 Wh is 1W x 3600sec = 3600J

$$energy/m^2 = E_{sol} * 24 * 3600 J/m^2 = E_{sol} * 24 Wh/m^2$$

---
1.1. Start by computing the daily energy in Wh produced by a 1m2 solar panel

a. Plot time-series of solar electric production in Caen and Tours from 1974 to 2023. Comment the evolution.

b. Plot boxplots of daily productions for both cities. Comment the box plot.

c. Remove outliers using the interquartile range. 

d. Plot an histogram of daily electricity production, after removal of outliers.

Watchout: remove all days for which a outlier is observed in Caen **or** Tours to keep the same number of observations.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv("Radiation.csv")   # Unités : W/m²
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

df["Caen"] = df["Caen"]*24*0.18*0.75   # Unités : Wh/m²
df["Tours"] = df["Tours"]*24*0.18*0.75

ax = df.plot(y=["Caen", "Tours"], x="DATE", ylabel="Daily energy [Wh/m²]", title="Solar electric production in Caen and Tours from 1974 to 2023")



The solar radiation of both cities cycles every year (makes sense, there's less sun in Winter in the northern hemishphere), but starting roughly in 2020 things get a little weird.

In [ ]:
df[["Caen", "Tours"]].plot(kind="box",title="Boxplots of daily production for Caen and Tours", ylabel="Daily energy [Wh/m²]")

In [ ]:
caen_q1 = df["Caen"].quantile(0.25)
caen_q3 = df["Caen"].quantile(0.75)

caen_iqr = caen_q3 - caen_q1

tours_q1 = df["Tours"].quantile(0.25)
tours_q3 = df["Tours"].quantile(0.75)

tours_iqr = tours_q3 - tours_q1

outliers = df[
    (df["Caen"] < caen_q1 - caen_iqr) | (df["Caen"] > caen_q3 + caen_iqr) |
    (df["Tours"] < tours_q1 - tours_iqr) | (df["Tours"] > tours_q3 + tours_iqr)
]

df = df.drop(outliers.index)
df[["Caen", "Tours"]].plot(kind="box", title="Boxplots of daily production without outliers", ylabel="Daily energy [Wh/m²]")

In [ ]:
fig,axes = plt.subplots(1,2)
axes[0].set_xlabel("Daily energy [Wh/m²]")
ax = df[["Caen", "Tours"]].hist(bins=150, ax = axes)
fig.suptitle("Histogram of daily production without outliers")


---
1.2. We want to compute monthly statistics of electricity solar production. Calculate for each city and for each month: 

1) the average daily production of electricity in Wh/m2

2) the median daily production of electricity in Wh/m2

3) the standard deviation daily production of electricity in Wh/m2

4) the 5% percentile of daily production of electricity in Wh/m2

5) the 95% percentile of daily production of electricity in Wh/m2

Report the results in one or two tables. 

Compare and comment these statistics!

In [ ]:
table = {'Caen': [df["Caen"].mean(), df["Caen"].median(), df["Caen"].std(), 
                  df["Caen"].quantile(0.05), df["Caen"].quantile(0.95)],
        'Tours': [df["Tours"].mean(), df["Tours"].median(), df["Tours"].std(),
                  df["Tours"].quantile(0.05), df["Tours"].quantile(0.95)]
        }

tableCaen = {'Mean':[], 'Median':[], 'Std':[], '5Perc':[], '95Perc':[]}
tableTours = {'Mean':[], 'Median':[], 'Std':[], '5Perc':[], '95Perc':[]}
for monthIndex in range(1,13):
    tableCaen['Mean'].append(df['Caen'][df['DATE'].dt.month == monthIndex].mean())
    tableCaen['Median'].append(df['Caen'][df['DATE'].dt.month == monthIndex].median())
    tableCaen['Std'].append(df['Caen'][df['DATE'].dt.month == monthIndex].std())
    tableCaen['5Perc'].append(df['Caen'][df['DATE'].dt.month == monthIndex].quantile(0.05))
    tableCaen['95Perc'].append(df['Caen'][df['DATE'].dt.month == monthIndex].quantile(0.95))
    tableTours['Mean'].append(df['Tours'][df['DATE'].dt.month == monthIndex].mean())
    tableTours['Median'].append(df['Tours'][df['DATE'].dt.month == monthIndex].median())
    tableTours['Std'].append(df['Tours'][df['DATE'].dt.month == monthIndex].std())
    tableTours['5Perc'].append(df['Tours'][df['DATE'].dt.month == monthIndex].quantile(0.05))
    tableTours['95Perc'].append(df['Tours'][df['DATE'].dt.month == monthIndex].quantile(0.95))
    

caenStats = pd.DataFrame(tableCaen, index=['J','F','M','A','M','J','J','A','S','O','N','D'])
ToursStats = pd.DataFrame(tableTours, index=['J','F','M','A','M','J','J','A','S','O','N','D'])
annualStats = pd.DataFrame(table, index=['Mean', 'Median', 'Std', '5% percentile', '95% percentile'])

print('Stats for Caen:\n',caenStats,'\n')
print('Stats for Tours :\n',ToursStats, '\n')
print('Annual Stats : \n', annualStats)
print("""
We observe that both mean and median are higher for Tours than Caen. 
However,  Tours's std is also higher, which could still explain the differences between both cities.
Moreover, 0.05 and 0.95 percentiles are near.
""")

---
## 2. Fit of distributions and hypothesis tests

---
2.1. We focus on the daily production of electricity in April. Retrieve the data for month of April, in Caen and Tours. 

 1) Fit Gamma and normal distributions by log-likelihood maximization to 
    daily production of electricity during April (Caen & Tours).
    
 2) Compute the 4 log-likelihoods and select the best model for each location (justify your answer).
 
 3) Compare on the same plot the empirical, the  gamma and normal pdf (the
    empirical pdf is an histogram of frequencies).
    
 4) Why is there 3 parameters in python for the Gamma pdf whereas there
    is only 2 in the distribution seen during lectures? 

Remark : set floc to -0.001 for the gamma.fit (to avoid troubles in case of null observations)


In [ ]:
import numpy as np
import scipy.stats as sc
from scipy.stats import norm
from scipy.stats import gamma


# Tri des données pour ne garder que celles en avril
energyCaenAvril = df['Caen'][df['DATE'].dt.month == 4]
energyToursAvril = df['Tours'][df['DATE'].dt.month == 4]


# 1. sFit des données triées par rapport aux lois Normale et Gamma
# Trouver les valeurs pour que cela fit avec la Normale et la Gamma en fonction de la distribution
muNormCaen, stdNormCaen = norm.fit(energyCaenAvril)
muNormTours, stdNormTours = norm.fit(energyToursAvril)
shapeGammaCaen, locGammaCaen, scaleGammaCaen = gamma.fit(energyCaenAvril, floc=-0.001)
shapeGammaTours, locGammaTours, scaleGammaTours = gamma.fit(energyToursAvril, floc=-0.001)

# Axe d'abcisse pour l'histogramme (à améliorer avec min et max des valeurs ?) / -600 pour la Normale
x = np.linspace(-600,1600,150)

# Calcul des courbes à partir de la pdf (--> intégrale = 1)
normCaen = norm.pdf(x,muNormCaen, stdNormCaen)
normTours = norm.pdf(x,muNormTours, stdNormTours)
gammaCaen = gamma.pdf(x,shapeGammaCaen,locGammaCaen,scaleGammaCaen)
gammaTours = gamma.pdf(x, shapeGammaTours,locGammaTours,scaleGammaTours)


fig, (ax1, ax2) = plt.subplots(2, figsize=(10,10))

# Plot des courbes pour Caen 
ax1.plot(x,normCaen, color="red", label="Normale Distribution")
ax1.plot(x, gammaCaen, color="green", label="Gamma Distribution")

# Plot de l'histogramme d'avril pour Caen
ax1.hist(energyCaenAvril, bins=40, density=True, label="Empirical Distribution")

# Affichage graphe Caen
ax1.title.set_text("Caen")
ax1.legend(loc="upper right")
ax1.set_xlim(-400,1600)
ax1.set_ylim(0,0.0025)
ax1.set_xlabel("Daily energy [Wh/m²]")
ax1.set_ylabel("Frequency")


# Plot des courbes pour Tours
ax2.plot(x, normTours, color="red", label="Normale Distribution")
ax2.plot(x, gammaTours, color="green", label="Gamma Distribution")


# Plot de l'histogramme d'avril pour Tours
ax2.hist(energyToursAvril, bins=40, density=True, label="Empirical Distribution")

# Affichage graphe Tours
ax2.title.set_text("Tours")
ax2.legend(loc="upper right")
ax2.set_xlim(-400,1600)
ax2.set_ylim(0,0.0025)
ax2.set_xlabel("Daily energy [Wh/m²]")
ax2.set_ylabel("Frequency")

fig.suptitle("Histogram of daily production in april from 1974 to 2023 without outliers")

plt.show()

# Calcul de l'erreur quadratique 
xEdgeCaen = np.histogram(energyCaenAvril, bins=149, density = True)[1]
xEdgeTours = np.histogram(energyToursAvril, bins=149, density = True)[1]
histoCaen = np.histogram(energyCaenAvril, bins=150, density = True)[0]
histoTours = np.histogram(energyToursAvril, bins=150, density = True)[0]

erreurNormCaen = np.sqrt(np.mean((norm.pdf(xEdgeCaen,muNormCaen, stdNormCaen) - histoCaen)**2))
erreurNormTours = np.sqrt(np.mean((norm.pdf(xEdgeTours,muNormTours, stdNormTours) - histoTours)**2))
erreurGammaCaen = np.sqrt(np.mean((gamma.pdf(xEdgeCaen,shapeGammaCaen,locGammaCaen,scaleGammaCaen) - histoCaen)**2))
erreurGammaTours = np.sqrt(np.mean((gamma.pdf(xEdgeTours,shapeGammaTours,locGammaTours,scaleGammaTours) - histoTours)**2))

print("--- Erreurs (RMSE) ---")
print('Erreur Normale Caen : \t',erreurNormCaen)
print('Erreur Gamma Caen : \t',erreurGammaCaen)
print('Erreur Normale Tours : \t',erreurNormTours)
print('Erreur Gamma Tours : \t',erreurGammaTours)

#3.
print("""
RMSE error is lower with Normal fit for both cities. 
On plots we see that both distributions are near, but the Normal fits better the final steep in
high values.
""")

# 4.
print("""The third parameter used by Gamma in Python is used to pitch the location of the distribution on the x-axis.
This is necessary as empirical datas may not begin at x=0, depending on the context. (Could begin at x < 0 !)
In fact, as we saw during lectures, Gamma is defined for positive-only values.
        """)



---

2.2. Check if the average daily production in April is the same in Caen and Tours. Let us recall that the null hypothesis is

$H_0$: $\mu_{Caen} = \mu_{Tours}$.

Take care to comment your conclusions. Are all assumptions required to perform this test sastisfied?

On considère deux échantillons (Caen et Tours) 
notés $X_1$, $X_2$ avec :
    \
    $X_1 = \{X_{1,1}, ..., X_{1,n}\}$ ∼ $N(\mu_1,\sigma_1²) \Rightarrow \bar{X_1}$ ~ $N(\mu_1, \frac{\sigma_1²}{n_1})$
    \
    $X_2 = \{X_{2,1}, ..., X_{2,n}\}$ ∼ $N(\mu_2,\sigma_2²) \Rightarrow \bar{X_2}$ ~ $N(\mu_2, \frac{\sigma_2²}{n_2})$
    \
    \
Comme les $X_i$ ~ $N(\mu,\sigma²)$, on a la propriété suivante: $\frac{\bar{X}-\mu}{\sqrt{\frac{S²}{n}}}$ ~ $t_{n-1}$ dont on utilisera l'adaptation pour deux 
populations. Ceci justifie le choix d'un test de Student.

$H_0 : \mu_{caen} - \mu_{tours} = 0$
\
$H_1 : \mu_{caen} - \mu_{tours} \neq 0$


In [ ]:
#df = pd.read_csv("Radiation.csv")
#df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

# Tri des données pour ne garder que celles en avril
#avrilCaen = df['Caen'][df['DATE'].dt.month == 4]
#avrilTours = df['Tours'][df['DATE'].dt.month == 4]

#energy_caen = avrilCaen*24*0.18*0.75
#energy_tours = avrilTours*24*0.18*0.75

std_caen = energyCaenAvril.std()    #Ici on a déjà retiré les outliers
std_tours = energyToursAvril.std()

#Check if it is correct or not.     A priori OK
n = energyCaenAvril.shape[0]
s_pool = np.sqrt(((n-1)*std_caen**2+(n-1)*std_tours**2)/(n+n-2))

# Calculate the T-test for the means of two independent samples of scores.
statistic, pvalue = sc.ttest_ind(energyCaenAvril, energyToursAvril, axis=0, equal_var=True)

print(f"statistic = {statistic} pvalue = {pvalue}")

alpha = 0.05

if(pvalue > alpha) :
    print("On ne rejette pas H0")
else :
    print("On rejette H0 car pvalue < alpha")

# j'obtiens un Tx plus grand que ds les attempts précédents (ma pval aussi du coup)
# mais pas suffisant pour accepter H_0
#Attention, au 2.3, on voit que les variances sont différentes. 
#Le test de comparaison de la moyenne ne fonctionne que si la variance est la même, on ne peut donc pas conclure ! 
#(D'où le point 2.4 et 2.5, donc c'est OK)

# note : en retirant les outliers j'obtiens ceci : Tx = -4.839401221201778, t_l = -1.9608043779604514, t_u = 1.960804377960451, pval = 1.3724062193154514e-06

---
2.3. Test the equality of variance of daily production in April at Caen & Tours?
$H_0$: $\sigma_{Caen}=\sigma_{Tours}$.


In [ ]:
#on calcule notre p_value
F = std_caen**2/std_tours**2
pvalStd = sc.f.cdf(F,n-1,n-1)
alpha = 0.05

print('Pval =', pvalStd)

if(pvalue > alpha) :
    print("On ne rejette pas H0")
else :
    print("On rejette H0 car pavalue < alpha")

---
2.4. Explain the Wilcoxon's test. What is the main advantage of this test compared to the Student's T test. Why is this useful in our project? 

The Wilcoxon's test is another statistic test to compare the similarity between two independant samples of datas. The main advantage of this test compared to the Student's test is that the variance of both samples can be different. We can't trust our Student'T test because the equal-variance assumption is not verified. So the Wilcoxon's test is usefull to determine if the electrity production in Caen and Tours are the same.

---
2.5. Apply the Wilcoxon test to distributions of daily productions in April, at Caen and Tours.  What can you conclude about the means of daily production in these 2 cities?


In [ ]:
#Calcul de la p-valeur
stats, pvalWilcoxon = sc.wilcoxon(energyCaenAvril, energyToursAvril)
alpha = 0.05

print('Pval Wilcoxon = ',pvalWilcoxon)

if(pvalWilcoxon > alpha) :
    print("On ne rejette pas H0")
else :
    print("On rejette H0 car pavlue < alpha")





---
## 3. Regression and forecasting 

---
3.1. Do we observe any trend in the yearly solar production of electricity over the considered period?
To answer this question: 

a. You will compute the average daily production (Wh/m2) during April from 1977 up to 2019 (included).

b. You get a time-series of 44 values for each city. Regress these values on the explanatory
variables X=(Year-1977). Don't forget to add a constant term and analyze results. 

c. Plot on the same graph, the predicted and the observed values.

d. Comment your results! 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sc
from scipy.stats import norm
from scipy.stats import gamma

df = pd.read_csv("Radiation.csv")
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

df["Caen"] = df["Caen"]*24*0.18*0.75
df["Tours"] = df["Tours"]*24*0.18*0.75



#Donnees d'Avril pour Caen et Tours entre 1977 et 2019 inclus 
dataApril = df[["Caen","Tours"]][(df["DATE"].dt.month == 4) & (df["DATE"].dt.year >= 1977) & (df["DATE"].dt.year <= 2019)]

#Ajout colonne année
dataApril["Year"] = df["DATE"].dt.year

#Liste des années reprises (1977-2019)
annee = np.arange(1977,2020)

#Moyenne des données pour chaque mois d'avril //J'en ai 43 et pas 44
moyenneAvril = dataApril.groupby(["Year", df["DATE"].dt.month])["Caen", "Tours"].mean().reset_index()
moyenneAvril = moyenneAvril[["Caen", "Tours"]] #on enleve les colonnes de l'annee et du mois
moyenneAvril = moyenneAvril[["Caen", "Tours"]].mean(axis=1) #moyenne de Caens et Tours

#Creation matrice avec dans la 1ere colonne des constances et dans la 2eme colonne les années
X = np.column_stack((np.ones(len(moyenneAvril)), annee))

#Régression linéaire pour trouver la fonction qui sert à prédire la moyenne
beta = np.linalg.lstsq(X, moyenneAvril, rcond=None)[0]

#on evalue la fonction sur chaque annee entre 1977 et 2019
eval = np.zeros(len(annee))
for i in range(len(annee)) :
    #dans la doc, beta[1] et beta[0] sont inversés mais ca donne plus les memes valeurs donc je suis perdu
    #Prediction pour l'année i
    eval[i] = beta[1] * (1977 + i) + beta[0]

#Affichage graph
plt.plot(annee, moyenneAvril, 'o', label="Observed value")
plt.plot(annee, eval, label="Predicted value")
plt.legend(loc="upper right")
plt.ylim(0, 1000)
plt.show()

#Reste à commenter et verifier si c'est la bonne régression linéaire


---
3.2. You want to design a model to forecast the solar electric production for the next day (location Caen only). You will work with data over the period 1977 to 2019. 

Let us denote by C(t) the production on day 't'. The model that we want to fit is called autoregressive and is defined as follows:

$$C(t) = \sum_{k=1}^{10} a_k C(t-k) $$

This model is common in time-series analysis and predicts the production of the next day with the  recent observations.

a. Split the dataset into a training set (1977 to 2010 included) and a validation set (2011 to 2019 included).

b.	Estimate this model with statsmodels on the training set. 

c.	How would you judge the quality of the predictive model? (Analyze statistics reported by statsmodel)

d.	Compute the Mean Absolute Error (MAE) between predicted and real consumptions (on the training set).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sc
from scipy.stats import norm
from scipy.stats import gamma
import statsmodels.api as sm

df = pd.read_csv("Radiation.csv")
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

#Récupération données
trainingSet = df.loc[(df["DATE"].dt.year >= 1977) & (df["DATE"].dt.year <= 2010)]
validationSet = df.loc[(df["DATE"].dt.year >= 2011) & (df["DATE"].dt.year <= 2019)]

trainingSet.reset_index(inplace=True)
validationSet.reset_index(inplace=True)


energy_training_set = trainingSet["Caen"]*24*0.18*0.75
energy_validation_set = validationSet["Caen"]*24*0.18*0.75

#ordre du modèle autorégressif (ici k = 10)
ordre = 10

#Estimation de notre modèle
res = sm.tsa.AutoReg(energy_training_set, lags=ordre).fit()

#13342 to 16611

#On prédit les données du preier jour de 2011 au dernier jour de 2019
predictions = res.model.predict(res.params, start=energy_validation_set.index.min() , end=energy_validation_set.index.max())

#Affichage graph : bonus
plt.plot(energy_training_set.index, energy_training_set, label="Training set")
plt.plot(12296+energy_validation_set.index, energy_validation_set, label="Validation set", color="blue")
plt.plot(12296+predictions.index, valeurFuture, label="Forecast values")

plt.legend()
plt.show()

---
3.3. Use this model on the test set to forecast the electric daily production.

a. Compare on a graph, the forecast to  real consumptions on the given period. 

b. Plot the errors of prediction. Are they acceptable?

c. Compute the MAE on the test set and the $R^2$. Is the forecast reliable?

In [ ]:
# code here
